# Libraries

In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
import sys
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore')

# Set device

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Set Hyper Parameters

In [3]:
SEED = 19901109
batch_size = 1
learning_rate = 0.001
nepochs = 10000
max_patience_count = 100
notebookName = "DualEncoderAttention"

torch.manual_seed(SEED)

# Set Output Path

In [4]:

PATH = Path(f"./models/{notebookName}")
if os.path.isdir(PATH):
    dir_list = os.listdir(PATH)
    num_files = 0
    while True:
        if os.path.isfile(str(PATH / f"{num_files}")):
            num_files += 1
        else:
            break
else:
    os.mkdir(PATH)
    num_files = 0
num_files = 2

# Load Dataset

In [5]:
data_dir = Path("../input/google-smartphone-decimeter-challenge/")
df_train_default = pd.read_pickle(str(data_dir / "gsdc_extract_train.pkl.gzip"))
df_test = pd.read_pickle(str(data_dir / "gsdc_extract_test.pkl.gzip"))

In [6]:
def CustomTrainValidSplit(df:pd.DataFrame, valid_size):
    phones = df['phone'].unique()
    
    valid_num = int(len(phones) * valid_size)
    train_num = len(phones) - valid_num
    
    indexes = np.array(range(len(phones)))
    indexes = np.random.choice(indexes, len(indexes))
    
    df_train = []
    for phone in phones[indexes[:train_num]]:
        df_train.append(df[df['phone'] == phone])
    df_train = pd.concat(df_train)
    
    df_valid = []
    for phone in phones[indexes[train_num:-1]]:
        df_valid.append(df[df['phone'] == phone])
    df_valid = pd.concat(df_valid)
    
    return df_train.reset_index().drop(columns = 'index'), df_valid.reset_index().drop(columns = 'index')
    
df_train, df_valid = CustomTrainValidSplit(df_train_default, valid_size = 0.1)
print(df_train.shape, df_valid.shape)
del df_train_default

(123509, 148) (10061, 148)


In [7]:
class TimeSeriseDataset(torch.utils.data.Dataset):
    def __init__(self, df, phys_features, stat_features, labels, train = False):
        self.phys_features = phys_features
        self.stat_features = stat_features
        self.labels = labels
        self.train = train
        self.phones =df['phone'].unique()
        self._len = len(self.phones)
        
        self.df = df
        
    def __len__(self):
        return self._len
    
    def __getitem__(self, idx):
        phys_features = self.phys_features
        stat_features = self.stat_features
        labels = self.labels
        phone = self.phones[idx]
        
        df = self.df[self.df['phone']==phone]
        
        phys = torch.Tensor(df[phys_features].values)
        stat = torch.Tensor(df[stat_features].values)
        if self.train:
            label = torch.Tensor(df[labels].values)
        else:
            label = torch.Tensor([])
        return phys, stat, label

In [8]:
phys_features = [
    'latDeg', 
    'lngDeg', 
    'heightAboveWgs84EllipsoidM',
    'dlatDeg',
    'dlngDeg',
    'dheight',
    'UncalGyroXRadPerSec',
    'UncalGyroYRadPerSec',
    'UncalGyroZRadPerSec',
    'DriftXRadPerSec',
    'DriftYRadPerSec',
    'DriftZRadPerSec',
    'UncalAccelXMps2',
    'UncalAccelYMps2',
    'UncalAccelZMps2',
    'BiasXMps2',
    'BiasYMps2',
    'BiasZMps2',
    'UncalMagXMicroT',
    'UncalMagYMicroT',
    'UncalMagZMicroT',
    'BiasXMicroT',
    'BiasYMicroT',
    'BiasZMicroT',
    'yawDeg',
    'rollDeg',
    'pitchDeg',
]
stat_features = [
    'GPS_L1', 
    'GPS_L5', 
    'GAL_E1', 
    'GAL_E5A', 
    'GLO_G1', 
    'BDS_B1I', 
    'BDS_B1C', 
    'BDS_B2A', 
    'QZS_J1', 
    'QZS_J5',
    'xSatPosM',
    'ySatPosM',
    'zSatPosM',
    'xSatVelMps',
    'ySatVelMps',
    'zSatVelMps',
]

labels = [
    't_latDeg', 
    't_lngDeg', 
    't_heightAboveWgs84EllipsoidM',
#     'courseDegree',
#     'hDop',
#     'vDop',
#     'speedMps'
         ]

In [9]:
train_data = TimeSeriseDataset(df_train, 
                           phys_features, stat_features, labels,
                           train = True)
valid_data = TimeSeriseDataset(df_valid, 
                           phys_features, stat_features, labels,
                           train = True)
test_data = TimeSeriseDataset(df_test, 
                           phys_features, stat_features, labels,
                           train = False)
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(valid_data, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False)

In [10]:
def torch_haversine(pred, true):

    lat1=pred[:,0] % 360
    lon1=pred[:,1] % 360
    lat2=true[:,0] % 360
    lon2=true[:,1] % 360

    lat1, lat2, lon1, lon2 = map(lambda x:x*np.pi/180, [lat1, lat2, lon1, lon2])

    dlat = (lat2 - lat1)
    dlon = (lon2 - lon1)

    a = torch.sin(dlat / 2.0)**2 + torch.cos(lat1) * torch.cos(lat2) * (torch.sin(dlon / 2.0)**2)
    c = 2 * torch.arcsin(a ** 0.5)

    dist = 6_367_000 * c

    return dist

def quantile_mean(dist):
    return (torch.quantile(dist, 0.5) + torch.quantile(dist, 0.95))/2
    
def gps_loss(predict:torch.Tensor, target:torch.Tensor):
    dist = torch_haversine(predict, target)

    loss = dist.mean()

    return loss

def gps_score(predict:torch.Tensor, target:torch.Tensor):
    dist = torch_haversine(predict, target)

    score = quantile_mean(dist)

    return score

def calc_loss_and_score(predicts, grounds):
    dist = []
    for pred, ground in zip(predicts, grounds):
        pred = pred.squeeze(0)
        ground = ground.squeeze(0)
        dist.append(torch_haversine(pred, ground))
    dist = torch.cat(dist, axis = 0)
    return dist.mean(), quantile_mean(dist)

In [11]:
class Encoder(nn.Module):
    def __init__(self, num_features, num_hiddens, num_layers, dropout = 0.1, device_ = 'cpu'):
        super().__init__()
        self.device = device_
        
        self.gru = nn.GRU(num_features, num_hiddens, num_layers, batch_first = True, dropout = dropout, bidirectional = True)
    def forward(self, x):
        x, h = self.gru(x)
        h = h.to(device)
        return x, h

In [12]:
class AttDecoder(nn.Module):
    def __init__(self, num_features, num_hiddens, num_layers, dropout = 0.1, device_ = 'cpu'):
        super().__init__()
        self.device = device_
        self.num_features = num_features
        self.num_hiddens = num_hiddens
        self.max_length = 10000
        
        self.attn = nn.Linear(3 + num_hiddens*2, self.max_length)
        self.comb = nn.Linear(3 + num_hiddens*2, num_features)
        
        self.gru = nn.GRU(num_features, num_hiddens, num_layers, batch_first = True, dropout = dropout, bidirectional = True)
    def forward(self, x, h0, x0):
        hd = h0.reshape(1, 1, self.num_hiddens * 2)
        c = torch.cat([x, hd], axis = 2).reshape(-1, 3 + self.num_hiddens * 2)
        w = self.attn(c)
        w = w[:,:x0.shape[1]]
        w = F.softmax(w).unsqueeze(1)
        applied = torch.bmm(w, x0)
        
        output = torch.cat([x, applied], axis = 2)
        output = self.comb(output)
        output = F.relu(output)
        
        output, h = self.gru(output, h0)
        h = h.to(device)
        return output, h, w

In [13]:
class BaseModel(nn.Module):
    def __init__(self, num_state, num_status, num_outputs, device_ = 'cpu'):
        super().__init__()
        self.device = device_
        self.encoder_hidden = 16
        
        self.phys_encoder = Encoder(num_state, self.encoder_hidden, 1, 0.6, device_)
        self.stat_encoder = Encoder(num_status, self.encoder_hidden, 1, 0.6, device_)
        self.decoder = AttDecoder(num_outputs, self.encoder_hidden*2, 1, 0.3, device_)
        
        self.conv = nn.Conv1d(self.encoder_hidden*4, 3, 1)
        
    def forward(self, phys, stat):
        phys_x, phys_h = self.phys_encoder(phys)
        stat_x, stat_h = self.stat_encoder(stat)

        h0 = torch.cat([phys_h, stat_h], axis = 2)
        x0 = torch.cat([phys_x, stat_x], axis = 2)
        
        out_ = torch.zeros(phys.shape[0], phys.shape[1], self.encoder_hidden * 4).to(self.device)
        hd = h0
        for i in range(phys.shape[1]):
            out_[:,i,:], hd, weights = self.decoder(phys[:,i,:3].unsqueeze(1), hd, x0)
            
        out = out_.transpose(1,2)
        out = self.conv(out)
        out = out.transpose(1,2)
        return out

In [14]:
model = BaseModel(len(phys_features), len(stat_features), len(labels), device)
model.to(device)
# model.load_state_dict(torch.load("./models/DualEncoderAttention/model-2_checkpoint/model-22-6455965.0.pth"))

optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                mode = 'min',
                                                factor = 0.1,
                                                patience = 5,
                                                verbose = True)

In [15]:
def train(epoch, progress_log):
    model.train()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    
    for phys, stat, label in progress_log:
        
        phys = phys.to(device)
        stat = stat.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()  # 경사를 0으로 초기화
        pred = model(phys, stat)  # 데이터를 입력하고 출력을 계산
        
        loss = gps_loss(pred, label)  # 출력과 훈련 데이터 정답 간의 오차를 계산
        score = gps_score(pred, label)  # 출력과 훈련 데이터 정답 간의 오차를 계산

        loss.backward()  # 오차를 역전파 계산
        optimizer.step()  # 역전파 계산한 값으로 가중치를 수정
        
        predict.append(pred)
        ground.append(label)
            
    loss, score = calc_loss_and_score(predict, ground)
    
    del predict, ground
    return loss, score

In [16]:
def valid(epoch, progress_log):
    model.eval()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    
    with torch.no_grad():
        for phys, stat, label in progress_log:

            phys = phys.to(device)
            stat = stat.to(device)
            label = label.to(device)

            pred = model(phys, stat)  # 데이터를 입력하고 출력을 계산

            predict.append(pred)
            ground.append(label)
            
    loss, score = calc_loss_and_score(predict, ground)
    
    del predict, ground
    return loss, score

In [17]:
def test(epoch, progress_log):
    model.eval()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    
    with torch.no_grad():
        for phys, stat, _ in progress_log:

            phys = phys.to(device)
            stat = stat.to(device)

            pred = model(phys, stat)  # 데이터를 입력하고 출력을 계산

            predict.append(pred.cpu())
    
    return predict

In [ ]:
train_loss_list = []
train_score_list = []
valid_loss_list = []
valid_score_list = []

patience_count = 0
min_valid_score = np.inf
checkpoint_name = ""

if not os.path.isdir(f"./models/{notebookName}/model-{num_files}_checkpoint/"):
    os.mkdir(f"./models/{notebookName}/model-{num_files}_checkpoint/")
    
prog_epoch = tqdm(range(0, nepochs), position = 0, desc = 'EPOCH')
for epoch in prog_epoch:
    prog_train = tqdm(train_loader, desc = 'TRAIN', leave = False)
    prog_valid = tqdm(valid_loader, desc = 'VALID', leave = False)
    
    train_loss, train_score = train(epoch, prog_train)
    valid_loss, valid_score = valid(epoch, prog_valid)
    
    scheduler.step(valid_score)
    
    if min_valid_score > valid_score:
        min_valid_score = valid_score
        checkpoint_name = f"./models/{notebookName}/model-{num_files}_checkpoint/model-{epoch}-{valid_score}.pth"
        torch.save(model.state_dict(), checkpoint_name)
    else:
        patience_count+=1
        if(patience_count > max_patience_count):
            break
        
    train_loss_list.append(train_loss)
    train_score_list.append(train_score)
    valid_loss_list.append(valid_loss)
    valid_score_list.append(valid_score)
    
    print( "-------------------------------------------------------")
    print(f"|EPOCH: {epoch+1}/{nepochs}")
    print(f"|TRAIN: loss={train_loss:.6f},  score={train_score:.6f}|")
    print(f"|VALID: loss={valid_loss:.6f},  score={valid_score:.6f}|")
    
    
history = dict()
history['train_loss'] = train_loss_list
history['train_score'] = train_score_list
history['valid_loss'] = valid_loss_list
history['valid_score'] = valid_score_list

EPOCH:   0%|          | 0/10000 [00:00<?, ?it/s]

TRAIN:   0%|          | 0/46 [00:00<?, ?it/s]

VALID:   0%|          | 0/6 [00:00<?, ?it/s]

-------------------------------------------------------
|EPOCH: 1/10000
|TRAIN: loss=12628969.000000,  score=12702318.000000|
|VALID: loss=12467236.000000,  score=12474848.000000|


TRAIN:   0%|          | 0/46 [00:00<?, ?it/s]

...

TRAIN:   0%|          | 0/46 [00:00<?, ?it/s]

VALID:   0%|          | 0/6 [00:00<?, ?it/s]

-------------------------------------------------------
|EPOCH: 3/10000
|TRAIN: loss=11837885.000000,  score=11940806.000000|
|VALID: loss=11579855.000000,  score=11592094.000000|


TRAIN:   0%|          | 0/46 [00:00<?, ?it/s]

VALID:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# Load submission sample
submission = pd.read_csv(str(data_dir / "sample_submission.csv"))
print(submission.shape)
submission.head()

In [ ]:
model.load_state_dict(torch.load(checkpoint_name))
torch.save(model.state_dict(), f"./models/{notebookName}/model-{min_valid_score}.pth")

In [ ]:
predict = test(0, tqdm(test_loader))

In [ ]:
predict = torch.cat(predict, axis = 1).squeeze(0)
submission['latDeg'] = predict[:,0]
submission['lngDeg'] = predict[:,1]

In [ ]:
submission.to_csv(f"./models/{notebookName}/result-{num_files}-{min_valid_score}.csv", index = False)
pd.DataFrame([]).to_csv(PATH / f"{num_files}")

# Reference
1. [pytorch tutorial-seq2seq_translation](https://tutorials.pytorch.kr/intermediate/seq2seq_translation_tutorial.html)